In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Update the file path as per your Drive location
file_path = '/content/drive/My Drive/CEAS_08.csv'

# Load the dataset
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())


                                              sender  \
0                   Young Esposito <Young@iworld.de>   
1                       Mok <ipline's1983@icable.ph>   
2  Daily Top 10 <Karmandeep-opengevl@universalnet...   
3                 Michael Parker <ivqrnai@pobox.com>   
4  Gretchen Suggs <externalsep1@loanofficertool.com>   

                                         receiver  \
0                     user4@gvc.ceas-challenge.cc   
1                   user2.2@gvc.ceas-challenge.cc   
2                   user2.9@gvc.ceas-challenge.cc   
3  SpamAssassin Dev <xrh@spamassassin.apache.org>   
4                   user2.2@gvc.ceas-challenge.cc   

                              date  \
0  Tue, 05 Aug 2008 16:31:02 -0700   
1  Tue, 05 Aug 2008 18:31:03 -0500   
2  Tue, 05 Aug 2008 20:28:00 -1200   
3  Tue, 05 Aug 2008 17:31:20 -0600   
4  Tue, 05 Aug 2008 19:31:21 -0400   

                                             subject  \
0                          Never agree to be a loser   
1  

In [ ]:
# Display dataset information
print(data.info())

# Display the first few rows
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39154 entries, 0 to 39153
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    39154 non-null  object
 1   receiver  38692 non-null  object
 2   date      39154 non-null  object
 3   subject   39126 non-null  object
 4   body      39154 non-null  object
 5   label     39154 non-null  int64 
 6   urls      39154 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.1+ MB
None
                                              sender  \
0                   Young Esposito <Young@iworld.de>   
1                       Mok <ipline's1983@icable.ph>   
2  Daily Top 10 <Karmandeep-opengevl@universalnet...   
3                 Michael Parker <ivqrnai@pobox.com>   
4  Gretchen Suggs <externalsep1@loanofficertool.com>   

                                         receiver  \
0                     user4@gvc.ceas-challenge.cc   
1                   user2.2@gvc.ceas-challenge.cc   

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import joblib


In [ ]:
# Features and labels
email_text_column = 'body'  # Email content
label_column = 'label'      # Labels

# Check for missing values in these columns
print(data[email_text_column].isnull().sum())
print(data[label_column].isnull().sum())

# Drop rows with missing values (if any)
data = data.dropna(subset=[email_text_column, label_column])

# Confirm the data is clean
print(data.info())


0
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39154 entries, 0 to 39153
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    39154 non-null  object
 1   receiver  38692 non-null  object
 2   date      39154 non-null  object
 3   subject   39126 non-null  object
 4   body      39154 non-null  object
 5   label     39154 non-null  int64 
 6   urls      39154 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.1+ MB
None


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and labels (y)
X = data[email_text_column]
y = data[label_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")


Training samples: 31323
Testing samples: 7831


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Create the pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Evaluate the model on the test data
y_pred = model.predict(X_test)
print("Classification Report:\n")
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3490
           1       1.00      0.98      0.99      4341

    accuracy                           0.99      7831
   macro avg       0.99      0.99      0.99      7831
weighted avg       0.99      0.99      0.99      7831



In [ ]:
import joblib

# Save the model
model_path = '/content/drive/My Drive/phishing_detector.pkl'
joblib.dump(model, model_path)

print(f"Model saved at: {model_path}")


Model saved at: /content/drive/My Drive/phishing_detector.pkl


In [ ]:
!pip install flask
!pip install flask pyngrok
# Load the saved model
loaded_model = joblib.load(model_path)

# Test with a sample email
sample_email = ["Your account has been compromised. Click this link to reset your password immediately."]
prediction = loaded_model.predict(sample_email)

# Output the result
print("Prediction (1 = phishing, 0 = legit):", prediction)


Prediction (1 = phishing, 0 = legit): [0]


In [ ]:

!pip install flask



In [ ]:
!pip list | grep flask


In [ ]:
import joblib
from flask import Flask, render_template, request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Load the pre-trained phishing detection model
model_path = '/content/drive/My Drive/phishing_detector.pkl'  # Make sure the model is saved in this location
model = joblib.load(model_path)

# Initialize the Flask app
app = Flask(__name__)

# Home route to display the form
@app.route('/')
def home():
    return render_template('index.html')

# Route to handle email submission and display result
@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Get the email content from the form
        email_content = request.form['email_content']

        # Predict if the email is phishing or not
        prediction = model.predict([email_content])[0]

        # Prepare result message
        result = "Phishing Email Detected!" if prediction == 1 else "This Email is Legitimate."

        return render_template('index.html', result=result, email_content=email_content)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install flask pyngrok


In [ ]:
!ngrok authtoken 2qY6P14s2PB68GkOfbe2AFsxMTX_4gPGGyxG1U2MtJCZJ8kwo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from flask import Flask, render_template_string, request
!pip install pyngrok
from pyngrok import ngrok
import joblib

# Load the pre-trained model
model_path = '/content/drive/My Drive/phishing_detector.pkl'  # Update with your model path
model = joblib.load(model_path)

# Initialize the Flask app
app = Flask(__name__)

# Define the HTML template with CSS for styling
html_code = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Phishing Email Detector</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Poppins', sans-serif;
            background-color: #f9fafc;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .container {
            background: linear-gradient(145deg, #ffffff, #e6e6e6);
            border-radius: 15px;
            box-shadow: 4px 4px 10px rgba(0, 0, 0, 0.1);
            width: 90%;
            max-width: 600px;
            padding: 30px;
            text-align: center;
        }
        h1 {
            color: #2d89ef;
            font-size: 2em;
        }
        p {
            font-size: 1.1em;
            color: #666;
        }
        textarea {
            width: 100%;
            height: 150px;
            padding: 12px;
            font-size: 1em;
            border-radius: 8px;
            border: 1px solid #ccc;
            outline: none;
            box-shadow: inset 2px 2px 5px #ddd;
        }
        textarea:focus {
            border-color: #2d89ef;
            box-shadow: inset 2px 2px 5px #bbb;
        }
        button {
            background-color: #2d89ef;
            color: white;
            padding: 12px 24px;
            border: none;
            border-radius: 8px;
            font-size: 1em;
            cursor: pointer;
            transition: background-color 0.3s;
            margin-top: 15px;
        }
        button:hover {
            background-color: #1a73e8;
        }
        .result {
            margin-top: 20px;
            font-size: 1.2em;
            padding: 10px;
            border-radius: 8px;
        }
        .phishing {
            color: #d32f2f;
            background-color: #fdecea;
        }
        .legitimate {
            color: #388e3c;
            background-color: #e8f5e9;
        }
        @media (max-width: 600px) {
            h1 {
                font-size: 1.8em;
            }
            textarea {
                height: 120px;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Phishing Email Detector</h1>
        <p>Enter the email content below and click "Analyze" to detect phishing emails.</p>

        <form action="/predict" method="POST">
            <textarea name="email_content" placeholder="Enter email content here..."></textarea><br>
            <button type="submit">Analyze</button>
        </form>

        <!-- Dynamic result display -->
        {% if result %}
        <div class="result {% if 'Phishing' in result %}phishing{% else %}legitimate{% endif %}">
            <h3>{{ result }}</h3>
        </div>
        {% endif %}
    </div>
</body>
</html>

'''

# Home route
@app.route('/')
def home():
    return render_template_string(html_code)

# Route to handle email submission and display result
@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Get the email content from the form
        email_content = request.form['email_content']

        # Predict if the email is phishing or not
        prediction = model.predict([email_content])[0]

        # Prepare result message
        result = "Phishing Email Detected!" if prediction == 1 else "This Email is Legitimate."

        return render_template_string(html_code, result=result, email_content=email_content)

# Run the app with ngrok tunnel
public_url = ngrok.connect(5000)
print("Flask app is running at:", public_url)

app.run(port=5000)


Flask app is running at: NgrokTunnel: "https://bc47-34-106-190-203.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:36:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:36:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:37:15] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:37:36] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:39:04] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:39:42] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:39:51] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:40:39] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:41:33] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 14:42:30] "POST /predi